

---

## 우리 데이터로 시각화 실험해보기
### 지역별 한부모 복지 시설의 유형 별 수 확인하기

### 환경설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 저장 시 폰트 깨지지 않도록 세팅

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (12.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120831 files and direc

In [3]:
import folium

In [4]:
# 최신 버전 다운 시 이미지 저장 오류 발생! 5.3.1로 다운해야함
! pip install plotly==5.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [5]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [6]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.6 MB/s eta 0:00:00


In [7]:
# 프로그램 다운로드 2 -> 코랩이 아닌 터미널 등 컴퓨터 자체에서 입력
# conda install -c conda-forge python-kaleido

---
여기까지 설치후 런타임 재시작 후 전체 재실행

### json 파일 불러오기

In [8]:
import requests
import json


r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [9]:
# json 파일 훑어보기
# 서울 행정구역 json raw파일(githubcontent) -> polygon data, 위도와 경도, 'code'로 구성되어 있음
# 강동구의 코드가 11250 => 확인 결과, 행정동 코드로 판명
seoul_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

### 총합 복지 시설 구하기

In [10]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
allData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용_최종.xlsx', sheet_name = '총합시설')

allData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,총합,23,2
1,중구,총합,11,2
2,종로구,총합,14,4
3,은평구,총합,28,15
4,용산구,총합,15,5
5,영등포구,총합,15,5
6,양천구,총합,27,17
7,송파구,총합,30,18
8,성북구,총합,27,12
9,성동구,총합,26,18


In [11]:
# 데이터 전처리 - 컬럼명 매칭
allData = allData.rename(columns={'시군구명':'name'})

#### 전체 복지 시설

In [12]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

allFig = px.choropleth(allData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
allFig.update_geos(fitbounds="locations", visible=False)
allFig.update_layout(title_text='서울시 구별 전체 복지 시설', title_font_size=20)

In [13]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
allFig.write_image('서울시 구별 전체 복지 시설.png')

In [14]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
allFig.write_html("서울시 구별 전체 복지 시설.html")

#### 입소 가능 전체 복지 시설

In [15]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

allUFig = px.choropleth(allData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
allUFig.update_geos(fitbounds="locations", visible=False)
allUFig.update_layout(title_text='서울시 구별 입소 가능 전체 복지 시설', title_font_size=20)

In [16]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
allUFig.write_image('서울시 구별 입소 가능 전체 복지 시설.png')

In [17]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
allUFig.write_html("서울시 구별 입소 가능 전체 복지 시설.html")

### 기타 시설

#### 기타 시설 - 전체 시설

In [18]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
etcData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '기타시설')

etcData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,기타시설,0,0
1,중구,기타시설,0,0
2,종로구,기타시설,0,0
3,은평구,기타시설,0,0
4,용산구,기타시설,0,0
5,영등포구,기타시설,0,0
6,양천구,기타시설,0,0
7,송파구,기타시설,0,0
8,성북구,기타시설,0,0
9,성동구,기타시설,0,0


In [19]:
# 데이터 전처리 - 컬럼명 매칭
etcData = etcData.rename(columns={'시군구명':'name'})

In [20]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

etcFig = px.choropleth(etcData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
etcFig.update_geos(fitbounds="locations", visible=False)
etcFig.update_layout(title_text='서울시 구별 기타 복지 시설', title_font_size=20)

In [21]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
etcFig.write_image('서울시 구별 기타 복지 시설.png')

In [22]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
etcFig.write_html("서울시 구별 기타 복지 시설.html")

#### 입소 가능 시설 수

In [23]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

etcUFig = px.choropleth(etcData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
etcUFig.update_geos(fitbounds="locations", visible=False)
etcUFig.update_layout(title_text='서울시 구별 입소 가능 기타 복지 시설', title_font_size=20)

In [24]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
etcUFig.write_image('서울시 구별 입소 가능 기타 복지 시설.png')

In [25]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
etcUFig.write_html("서울시 구별 입소 가능 기타 복지 시설.html")

### 자활 시설

#### 자활 시설 - 전체 시설

In [26]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
supData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '자활시설')

supData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,자활시설,1,0
1,중구,자활시설,1,0
2,종로구,자활시설,1,0
3,은평구,자활시설,1,0
4,용산구,자활시설,1,1
5,영등포구,자활시설,1,0
6,양천구,자활시설,2,1
7,송파구,자활시설,1,1
8,성북구,자활시설,1,0
9,성동구,자활시설,1,1


In [27]:
# 데이터 전처리 - 컬럼명 매칭
supData = supData.rename(columns={'시군구명':'name'})

In [28]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

supFig = px.choropleth(supData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
supFig.update_geos(fitbounds="locations", visible=False)
supFig.update_layout(title_text='서울시 구별 자활 시설', title_font_size=20)

In [29]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
supFig.write_image('서울시 구별 자활 시설.png')

In [30]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
supFig.write_html("서울시 구별 자활 시설.html")

#### 입소 가능 시설 수

In [31]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

supUFig = px.choropleth(supData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
supUFig.update_geos(fitbounds="locations", visible=False)
supUFig.update_layout(title_text='서울시 구별 입소 가능 자활 시설', title_font_size=20)

In [32]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
supUFig.write_image('서울시 구별 입소 가능 자활 시설.png')

In [33]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
supUFig.write_html("서울시 구별 입소 가능 자활 시설.html")

### 일반 사회 복지 시설

#### 일반 사회 복지 시설 - 전체 시설

In [34]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
norData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '일반사회복지시설')

norData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,일반사회복지시설,4,0
1,중구,일반사회복지시설,3,0
2,종로구,일반사회복지시설,1,0
3,은평구,일반사회복지시설,3,0
4,용산구,일반사회복지시설,2,0
5,영등포구,일반사회복지시설,2,0
6,양천구,일반사회복지시설,5,1
7,송파구,일반사회복지시설,6,0
8,성북구,일반사회복지시설,5,0
9,성동구,일반사회복지시설,3,0


In [35]:
# 데이터 전처리 - 컬럼명 매칭
norData = norData.rename(columns={'시군구명':'name'})
norData

,name,시설종류상세,전체시설수,입소가능시설수
0,중랑구,일반사회복지시설,4,0
1,중구,일반사회복지시설,3,0
2,종로구,일반사회복지시설,1,0
3,은평구,일반사회복지시설,3,0
4,용산구,일반사회복지시설,2,0
5,영등포구,일반사회복지시설,2,0
6,양천구,일반사회복지시설,5,1
7,송파구,일반사회복지시설,6,0
8,성북구,일반사회복지시설,5,0
9,성동구,일반사회복지시설,3,0


In [36]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

norFig = px.choropleth(norData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
norFig.update_geos(fitbounds="locations", visible=False)
norFig.update_layout(title_text='서울시 구별 일반 사회복지 시설', title_font_size=20)

In [37]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
norFig.write_image('서울시 구별 일반 사회복지 시설.png')

In [38]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
norFig.write_html("서울시 구별 일반 사회복지 시설.html")

#### 입소 가능 시설 수

In [39]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

norUFig = px.choropleth(norData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
norUFig.update_geos(fitbounds="locations", visible=False)
norUFig.update_layout(title_text='서울시 구별 입소 가능 일반 사회복지 시설', title_font_size=20)

In [40]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
norUFig.write_image('서울시 구별 입소 가능 일반 사회복지 시설.png')

In [41]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
norUFig.write_html("서울시 구별 입소 가능 일반 사회복지 시설.html")

### 아동 복지 시설

#### 아동복지 시설 - 전체 시설

In [42]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
kidData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '아동복지시설')

kidData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,아동복지시설,17,2
1,중구,아동복지시설,5,2
2,종로구,아동복지시설,10,3
3,은평구,아동복지시설,22,14
4,용산구,아동복지시설,10,2
5,영등포구,아동복지시설,11,4
6,양천구,아동복지시설,18,15
7,송파구,아동복지시설,21,17
8,성북구,아동복지시설,18,10
9,성동구,아동복지시설,20,16


In [43]:
# 데이터 전처리 - 컬럼명 매칭
kidData = kidData.rename(columns={'시군구명':'name'})
kidData

,name,시설종류상세,전체시설수,입소가능시설수
0,중랑구,아동복지시설,17,2
1,중구,아동복지시설,5,2
2,종로구,아동복지시설,10,3
3,은평구,아동복지시설,22,14
4,용산구,아동복지시설,10,2
5,영등포구,아동복지시설,11,4
6,양천구,아동복지시설,18,15
7,송파구,아동복지시설,21,17
8,성북구,아동복지시설,18,10
9,성동구,아동복지시설,20,16


In [44]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

kidFig = px.choropleth(kidData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
kidFig.update_geos(fitbounds="locations", visible=False)
kidFig.update_layout(title_text='서울시 구별 아동복지 시설', title_font_size=20)

In [45]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
kidFig.write_image('서울시 구별 아동복지 시설.png')

In [46]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
kidFig.write_html("서울시 구별 아동복지 시설.html")

#### 입소 가능 시설 수

In [47]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

kidUFig = px.choropleth(kidData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
kidUFig.update_geos(fitbounds="locations", visible=False)
kidUFig.update_layout(title_text='서울시 구별 입소 가능 아동복지 시설', title_font_size=20)

In [48]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
kidUFig.write_image('서울시 구별 입소 가능 아동복지 시설.png')

In [49]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
kidUFig.write_html("서울시 구별 입소 가능 일반 사회복지 시설.html")

### 건강가정지원센터

#### 건강가정 지원센터 - 전체 시설

In [50]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
famData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '건강가정지원센터')

famData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,건강가정지원센터,1,0
1,중구,건강가정지원센터,2,0
2,종로구,건강가정지원센터,1,0
3,은평구,건강가정지원센터,1,0
4,용산구,건강가정지원센터,0,0
5,영등포구,건강가정지원센터,0,0
6,양천구,건강가정지원센터,1,0
7,송파구,건강가정지원센터,1,0
8,성북구,건강가정지원센터,1,0
9,성동구,건강가정지원센터,1,0


In [51]:
# 데이터 전처리 - 컬럼명 매칭
famData = famData.rename(columns={'시군구명':'name'})
famData

,name,시설종류상세,전체시설수,입소가능시설수
0,중랑구,건강가정지원센터,1,0
1,중구,건강가정지원센터,2,0
2,종로구,건강가정지원센터,1,0
3,은평구,건강가정지원센터,1,0
4,용산구,건강가정지원센터,0,0
5,영등포구,건강가정지원센터,0,0
6,양천구,건강가정지원센터,1,0
7,송파구,건강가정지원센터,1,0
8,성북구,건강가정지원센터,1,0
9,성동구,건강가정지원센터,1,0


In [52]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

famFig = px.choropleth(famData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
famFig.update_geos(fitbounds="locations", visible=False)
famFig.update_layout(title_text='서울시 구별 건강가정 지원센터', title_font_size=20)

In [53]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
famFig.write_image('서울시 구별 건강가정 지원센터.png')

In [54]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
famFig.write_html("서울시 구별 건강가정 지원센터.html")

#### 입소 가능 시설 수

In [55]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

famUFig = px.choropleth(famData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
famUFig.update_geos(fitbounds="locations", visible=False)
famUFig.update_layout(title_text='서울시 구별 입소 가능 건강가정 지원센터', title_font_size=20)

In [56]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
famUFig.write_image('서울시 구별 입소 가능 건강가정 지원센터.png')

In [57]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
famUFig.write_html("서울시 구별 입소 가능 건강가정 지원센터.html")

### 한부모 가족 복지 시설

#### 한부모가족 복지시설 - 전체 시설

In [58]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
oneData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/사회복지시설_시각화용.xlsx', sheet_name = '한부모가족복지시설')

oneData

,시군구명,시설종류상세,전체시설수,입소가능시설수
0,중랑구,한부모가족복지시설,0,0
1,중구,한부모가족복지시설,0,0
2,종로구,한부모가족복지시설,1,1
3,은평구,한부모가족복지시설,1,1
4,용산구,한부모가족복지시설,2,2
5,영등포구,한부모가족복지시설,1,1
6,양천구,한부모가족복지시설,1,0
7,송파구,한부모가족복지시설,1,0
8,성북구,한부모가족복지시설,2,2
9,성동구,한부모가족복지시설,1,1


In [59]:
# 데이터 전처리 - 컬럼명 매칭
oneData = oneData.rename(columns={'시군구명':'name'})
oneData

,name,시설종류상세,전체시설수,입소가능시설수
0,중랑구,한부모가족복지시설,0,0
1,중구,한부모가족복지시설,0,0
2,종로구,한부모가족복지시설,1,1
3,은평구,한부모가족복지시설,1,1
4,용산구,한부모가족복지시설,2,2
5,영등포구,한부모가족복지시설,1,1
6,양천구,한부모가족복지시설,1,0
7,송파구,한부모가족복지시설,1,0
8,성북구,한부모가족복지시설,2,2
9,성동구,한부모가족복지시설,1,1


In [60]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

oneFig = px.choropleth(oneData, geojson=seoul_geo, locations='name', color='전체시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
oneFig.update_geos(fitbounds="locations", visible=False)
oneFig.update_layout(title_text='서울시 구별 한부모가족 복지시설', title_font_size=20)

In [61]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
oneFig.write_image('서울시 구별 한부모가족 복지시설.png')

In [62]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
oneFig.write_html("서울시 구별 한부모가족 복지시설.html")

#### 입소 가능 시설 수

In [63]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

oneUFig = px.choropleth(oneData, geojson=seoul_geo, locations='name', color='입소가능시설수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
oneUFig.update_geos(fitbounds="locations", visible=False)
oneUFig.update_layout(title_text='서울시 구별 입소 가능 한부모가족 복지시설', title_font_size=20)

In [64]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/figDatas')
oneUFig.write_image('서울시 구별 입소 가능 한부모가족 복지시설.png')

In [65]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
oneUFig.write_html("서울시 구별 입소 가능 한부모가족 복지시설.html")